## Deliverable 2. Create a Customer Travel Destinations Map.

In [80]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,52.66,58,98,4.00,overcast clouds
1,1,Guhagar,IN,17.4842,73.1929,73.58,92,98,2.84,overcast clouds
2,2,Talnakh,RU,69.4865,88.3972,-23.80,98,100,11.23,overcast clouds
3,3,Khatanga,RU,71.9667,102.5000,-19.01,98,100,14.36,light snow
4,4,Castro,BR,-24.7911,-50.0119,70.00,70,53,9.26,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like? '))
max_temp = float(input('What is the maximum temperature you would like? '))

What is the minimum temperature you would like? 65
What is the maximum temperature you would like? 90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)
]

In [29]:
# 4a. Determine if there are any empty rows.
filtered_data_has_nans = any(filtered_cities_df.isna().sum() > 0)

In [33]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if filtered_data_has_nans:
    filtered_cities_df = filtered_cities_df.dropna()
clean_df = filtered_cities_df

In [71]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Guhagar,IN,73.58,overcast clouds,17.4842,73.1929,
4,Castro,BR,70.00,broken clouds,-24.7911,-50.0119,
5,Butaritari,KI,82.47,broken clouds,3.0707,172.7902,
10,Mangrol,IN,70.86,clear sky,21.1167,70.1167,
14,Saint George,US,73.87,clear sky,37.1041,-113.5841,
15,Hithadhoo,MV,81.34,clear sky,-0.6000,73.0833,
17,Tautira,PF,84.22,few clouds,-17.7333,-149.1500,
20,Camopi,GF,74.53,overcast clouds,3.1655,-52.3158,
21,Avera,US,80.71,few clouds,33.1940,-82.5271,
25,Bubaque,GW,82.09,overcast clouds,11.2833,-15.8333,


In [72]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except(IndexError):
        print('Hotel not found... skipping.')    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [79]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == ''].index)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Guhagar,IN,73.58,overcast clouds,17.4842,73.1929,Hotel Seawinds
4,Castro,BR,70.00,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
5,Butaritari,KI,82.47,broken clouds,3.0707,172.7902,Isles Sunset Lodge
10,Mangrol,IN,70.86,clear sky,21.1167,70.1167,Hotel The Royal Honours
14,Saint George,US,73.87,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
...,...,...,...,...,...,...,...
690,Olinda,BR,82.13,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda
693,Hobyo,SO,78.48,few clouds,5.3505,48.5268,Halane Hotel and Restaurent
694,Jasper,US,72.27,scattered clouds,37.2001,-94.3502,Guest House Motel
696,Lindi,TZ,74.26,clear sky,-10.0000,39.7167,Lindi paradise house


In [81]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join('Vacation_Search','WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [83]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt><b>Hotel Name</b></dt>
        <dd>{Hotel Name}</dd>
    <dt><b>City</b></dt>
        <dd>{City}</dd>
    <dt><b>Country</b></dt>
        <dd>{Country}</dd>
    <dt><b>Current Weather</b></dt>
        <dd>{Current Description} and {Max Temp} \N{DEGREE SIGN}F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [86]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30, 31.0), zoom_level = 1.5)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))